In [ ]:
!pip install Cython numpy pandas pymystem3 sklearn tqdm
!pip install git+https://github.com/lopuhin/python-adagram.git
!pip install pymorphy2
!git clone https://github.com/nlpub/russe-wsi-kit.git
!pip install nltk

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-w4n0g23r
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-w4n0g23r
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=460243 sha256=e9370af61160876b2ad58a180edb89b01d34fa8db63e23983164fb02b59627f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-rzqedjww/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram
Cloning into 'russe-wsi-kit'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 122 (delta 0), reused 3 (delta 0), pack-reused 116
Receiving objects: 100% (122/122), 2.33 MiB | 18.53 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
!wget 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'


--2020-11-20 17:22:37--  https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.185.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.185.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462416741 (1.4G) [application/x-www-form-urlencoded]
Saving to: ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’

all.a010.p10.d300.w 100%[===================>]   1.36G  34.6MB/s    in 34s     

2020-11-20 17:23:12 (40.7 MB/s) - ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’ saved [1462416741/1462416741]



## Try baseline model

In [ ]:
from __future__ import print_function
import argparse
import os.path
import re
import sys
import warnings
import adagram
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_rand_score
import tqdm
import nltk
from nltk.corpus import stopwords
import pymorphy2

In [ ]:
model_path = 'all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
model = adagram.VectorModel.load(model_path)

In [ ]:
data_path = 'russe-wsi-kit/data/main/active-dict/test.csv'

In [ ]:
df = pd.read_csv(data_path, sep='\t')

In [ ]:
stop_words = nltk.download('stopwords')
stop_words = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()


def disambiguate(model, word, context):
    word, = lemmatized_context(word)
    if word == 'ребёнок':
      word = "ребенок"
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


def lemmatized_context(s):
  emb = [morph.parse(word)[0].normal_form.lower() for word in s.split(' ') if  re.match('[\w\-]+$', word)]
  without_stop_words = [i for i in emb if ( i not in stop_words)]
  return  without_stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['predict_sense_id'] = [disambiguate(model, word, context) for word, context in tqdm.tqdm(zip(df['word'], df['context']),
                                                                                            total=len(df))]

  0%|          | 0/3729 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
100%|██████████| 3729/3729 [00:11<00:00, 329.97it/s]


In [ ]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,2074,давление,NaN,3,0-9,Давление пара создается движением поршня в цил...
1,2075,давление,NaN,1,13-22,"«У тебя что, давление поднялось?» Я сказал, чт..."
2,2076,давление,NaN,1,56-65,Я жалуюсь Никоновичу наконец на головокружение...
3,2077,давление,NaN,1,0-9,Давление в котле не менялось
4,2078,давление,NaN,1,25-34,Он каждые два часа мерил давление и сахар в крови
...,...,...,...,...,...,...
3724,5798,зуд,NaN,2,43-47,Многих американцев одолевает романтический зуд...
3725,5799,зуд,NaN,3,23-27,Если на нее не находил зуд рассказывания истор...
3726,5800,зуд,NaN,3,27-33,"С раздражающей завистью, с зудом неудовлетворе..."
3727,5801,зуд,NaN,2,12-16,Нестерпимый зуд любопытства


# Save model

In [ ]:
save_path = 'russe-wsi-kit/data/main/active-dict/test_with_stop_baseline_adagrad.csv'
df.to_csv(save_path, sep='\t', index=None)

In [ ]:
cd russe-wsi-kit/data/main/active-dict

/content/russe-wsi-kit/data/main/active-dict


In [ ]:
!zip active-dict.zip test_with_stop_baseline_adagrad.csv

  adding: test_with_stop_baseline_adagrad.csv (deflated 68%)


In [ ]:
cd /content

/content


# Test model

In [ ]:
per_word = df.groupby('word').aggregate(lambda f: adjusted_rand_score(f.gold_sense_id, f.predict_sense_id))
per_word_ari = per_word['predict_sense_id']

ValueError: ignored

In [ ]:
for word, ari in zip(per_word.index, per_word_ari):
  print('{:<20} {:+.4f}'.format(word, ari))
print('Mean word ARI: {:.4f}'.format(np.mean(per_word_ari)))
